# Распаковка

In [2]:
import zipfile
from tqdm import tqdm
import os
path_to_zip_file = 'D:/Diana/Diploma/Buckeye_zip'
directory_to_extract_to = 'D:/Diana/Diploma/Buckeye_temp'
directory_to_extract_to_f = 'D:/Diana/Diploma/Buckeye'

In [3]:
zip_files = [os.path.join(path_to_zip_file, i) for i in os.listdir(path_to_zip_file)]

In [7]:
zip_files[0]

'D:/Diana/Diploma/Buckeye_zip\\s01.zip'

'D:/Diana/Diploma/Buckeye_zip\\s01.zip'

In [9]:
for r in tqdm(zip_files):
    with zipfile.ZipFile(r, 'r') as zip_ref:
        zip_ref.extractall(directory_to_extract_to)

100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [03:49<00:00,  5.73s/it]



In [10]:
pp = [os.path.join(directory_to_extract_to, i) for i in os.listdir(directory_to_extract_to)]

In [11]:
os.listdir(pp[0])

['s0101a.zip', 's0101b.zip', 's0102a.zip', 's0102b.zip', 's0103a.zip']

['s0101a.zip', 's0101b.zip', 's0102a.zip', 's0102b.zip', 's0103a.zip']

In [ ]:
files_to_zip = []
for i in tqdm(pp):
    for r in os.listdir(i):
        r_rep = r.replace('.zip', '')
        rr = os.path.join(i, r)
        directory_to_extract_to_fin = os.path.join(directory_to_extract_to_f, r_rep)
        os.makedirs(directory_to_extract_to_fin, exist_ok = True)
        
        with zipfile.ZipFile(rr, 'r') as zip_ref:
            zip_ref.extractall(directory_to_extract_to_fin)
            

In [ ]:
r = os.listdir(pp[0])[0]
r_rep = r.replace('.zip', '')
rr = os.path.join(pp[0], r)

In [ ]:
rr, r, r_rep

# Нарезка аудио файлов

In [ ]:
def make_lists_files(times, frames, phonemes, sr=16000, patterns = ['IVER', 'VOCNOISE', 'SIL']):
    nums = []
    for num in range(len(times)):
        if max(list(map(lambda x: x in phonemes[num], patterns))):
            nums.append(num)
    min_num = min(nums)

    audio_files = []
    times_new = []
    frames_new = []
    phonemes_new = []
    frames_new_cut = []

    start_frame = 0
    start_time = 0

    ff = []
    tt = []
    phph = []
    ff_cut = []

    for num in range(len(times)):
        if num >= min_num:
            if max(list(map(lambda x: x in phonemes[num], patterns))):
                if len(tt) > 0:
                    times_new.append(tt)
                if len(ff) > 0:
                    frames_new.append(ff)
                if len(phph) > 0:
                    phonemes_new.append(phph)
                if len(ff_cut) > 0:
                    frames_new_cut.append(ff_cut)

                ff = []
                tt = []
                phph = []

                start_frame = int(frames[num])
                start_time = times[num]

            else:
                ff.append(int(frames[num]))
                ff_cut.append(int(frames[num]) - int(start_frame))
                tt.append(times[num] - start_time)
                phph.append(phonemes[num])

    frames_to_cut = [[min(i), max(i)] for i in frames_new]

    return times_new, frames_new, phonemes_new, frames_to_cut,  # frames_new_cut

In [ ]:
def save_audio(wav_path, audio, frames_to_cut, sr = 16000, to_repl = '_fin'):
    for num in range(len(frames_to_cut)):
        min_frame = frames_to_cut[num][0]
        max_frame = frames_to_cut[num][1]
        aud_frame = audio[min_frame:max_frame]

        audio_filepath = wav_path.replace('Buckeye', 'Buckeye'+to_repl).replace('.wav', '_'+str(num)+'.wav')
        wavfile.write(audio_filepath, sr, aud_frame)

In [ ]:
def save_text(wav_path, frames_to_cut, times_new, phonemes_new, postfix = '', to_repl = '_fin'):
    lines = []
    for num in range(len(frames_to_cut)):
        tt = times_new[num]
        phph = phonemes_new[num]
        ll = []
        for num1 in range(len(tt)):
            ll.append(str(tt[num1]) + ' ' + phph[num1])
        lines.append(ll)

        text_filepath = wav_path.replace('Buckeye', 'Buckeye'+to_repl).replace('.wav', '_'+str(num)+postfix+'.txt')
        with open(text_filepath, 'w') as f:
            f.write('\n'.join(ll))

In [ ]:
fin_path = 'D:/Diana/Diploma/Buckeye_fin'

In [ ]:
ll = list(iter_find_files(buckeye_path, "*.wav"))

In [ ]:
wav_path = ll[1]

In [ ]:
not_done

In [ ]:
not_done = []
patterns = ['IVER', 'VOCNOISE', 'SIL']

for wav_path in tqdm(ll):
#     print(wav_path)
    
    phn_path = wav_path.replace(".wav", ".phones")
    words_path = wav_path.replace(".wav", ".words")

    new_path = wav_path.split('\\')[0].replace('Buckeye', 'Buckeye_fin')
    os.makedirs(new_path, exist_ok=True)

    with open(phn_path, "r") as f:
        lines = [i.strip() for i in f.readlines()]
    lines = lines[lines.index('#')+1:]
    lines = [i for i in lines if len(i) > 0]

    with open(words_path, "r") as f:
        words_lines = [i.strip() for i in f.readlines()]
    words_lines = words_lines[words_lines.index('#')+1:]
    words_lines = [i for i in words_lines if len(i) > 0 and i[0] != ';']

    sr, audio = wavfile.read(wav_path)
    times = [eval(i.split()[0]) for i in lines]
    frames = [eval(i.split()[0])*sr for i in lines]
    phonemes = [i.split()[-1] for i in lines]

    words_times = [eval(i.split()[0]) for i in words_lines]
    words_frames = [eval(i.split()[0])*sr for i in words_lines]
    words_phonemes = [i.split()[2].replace(';', '') for i in words_lines]

    nums = []
    for num in range(len(times)):
        if max(list(map(lambda x: x in phonemes[num], patterns))):
            nums.append(num)
    if len(nums)> 0:

        times_new, frames_new, phonemes_new, frames_to_cut = make_lists_files(
            times, frames, phonemes, patterns=patterns)
        times_new_words, frames_new_words, phonemes_new_words, frames_to_cut_words = make_lists_files(words_times,
                                                                                                      words_frames,
                                                                                                      words_phonemes)

        save_audio(wav_path, audio, frames_to_cut, sr=sr, to_repl = '_fin1')
        save_text(wav_path, frames_to_cut, times_new, phonemes_new, to_repl = '_fin1')
        save_text(wav_path, frames_to_cut_words, times_new_words,
                  phonemes_new_words, postfix='_words', to_repl = '_fin1')
    else:
        not_done.append(wav_path)

In [ ]:
not_done = []
patterns = ['IVER']

for wav_path in tqdm(ll):
#     print(wav_path)
    
    phn_path = wav_path.replace(".wav", ".phones")
    words_path = wav_path.replace(".wav", ".words")

    new_path = wav_path.split('\\')[0].replace('Buckeye', 'Buckeye_iver')
    os.makedirs(new_path, exist_ok=True)

    with open(phn_path, "r") as f:
        lines = [i.strip() for i in f.readlines()]
    lines = lines[lines.index('#')+1:]
    lines = [i for i in lines if len(i) > 0]

    with open(words_path, "r") as f:
        words_lines = [i.strip() for i in f.readlines()]
    words_lines = words_lines[words_lines.index('#')+1:]
    words_lines = [i for i in words_lines if len(i) > 0 and i[0] != ';']

    sr, audio = wavfile.read(wav_path)
    times = [eval(i.split()[0]) for i in lines]
    frames = [eval(i.split()[0])*sr for i in lines]
    phonemes = [i.split()[-1] for i in lines]

    words_times = [eval(i.split()[0]) for i in words_lines]
    words_frames = [eval(i.split()[0])*sr for i in words_lines]
    words_phonemes = [i.split()[2].replace(';', '') for i in words_lines]

    nums = []
    for num in range(len(times)):
        if max(list(map(lambda x: x in phonemes[num], patterns))):
            nums.append(num)
    if len(nums)> 0:

        times_new, frames_new, phonemes_new, frames_to_cut = make_lists_files(
            times, frames, phonemes, patterns=patterns)
        times_new_words, frames_new_words, phonemes_new_words, frames_to_cut_words = make_lists_files(words_times,
                                                                                                      words_frames,
                                                                                                      words_phonemes)

        save_audio(wav_path, audio, frames_to_cut, sr=sr, to_repl = '_iver')
        save_text(wav_path, frames_to_cut, times_new, phonemes_new, to_repl = '_iver')
        save_text(wav_path, frames_to_cut_words, times_new_words,
                  phonemes_new_words, postfix='_words', to_repl = '_iver')
    else:
        not_done.append(wav_path)

# Удаление пустых файлов

In [ ]:
from boltons.fileutils import iter_find_files
import torchaudio
from tqdm import tqdm
import os

In [ ]:
path = 'D:/Diana/Diploma/Buckeye_fin'
data = list(iter_find_files(path, "*.wav"))

In [ ]:
len(data)

In [ ]:
zero = []
for wav_path in tqdm(data):
    audio, sr = torchaudio.load(wav_path)
    audio = audio[0]
    audio_len = len(audio)
    if audio_len == 0:
        zero.append(wav_path)

In [ ]:
for wav_path in tqdm(zero):
    phn_path = wav_path.replace(".wav", ".txt")
    phn_path1 = wav_path.replace(".wav", "_words.txt")
    
    if os.path.isfile(wav_path):
        os.remove(wav_path)
    if os.path.isfile(phn_path):
        os.remove(phn_path)
    if os.path.isfile(phn_path1):
        os.remove(phn_path1)